## Scraping AirBnb  Web

### Import Libraries

In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import requests
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options

from selenium.webdriver.common.by import By
import time
import random as ran

### Scraping

In [2]:
def download_html(url):
    url = quote(url, safe=':/?&=')
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    response.close()
    return html

In [3]:
def get_driver(url, waiting_time):
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--headless=new')
    options.add_argument('--disable-dev-shm-usage')        
    options.add_argument('--blink-settings=imagesEnabled=false')
    driver = webdriver.Chrome(options=options)

    # options = Options()
    # options.headless = True  # Chế độ headless (không có giao diện người dùng)
    # # options.set_preference('permissions.default.image', 2)  # Tắt hình ảnh

    # driver = webdriver.Firefox(options=options)

    for i in range(10):
        try:
            driver.get(url)
            break
        except:
            pass
    time.sleep(waiting_time)
    return driver


In [4]:
def download_dynamic_html(url):
        for i in range(10):
                try:
                        driver = get_driver(url,20)
                        break
                except:
                        pass

        pageSource = driver.page_source
        driver.quit()
        return pageSource


In [5]:
def move_page(url,page):
    html = download_dynamic_html(url)
    soup = BeautifulSoup(html, 'html.parser')

    elements = soup.find_all('a',{'class':'l1ovpqvx atm_1y33qqm_1ggndnn_10saat9 atm_17zvjtw_zk357r_10saat9 atm_w3cb4q_il40rs_10saat9 c1ackr0h atm_c8_fkimz8 atm_g3_11yl58k atm_fr_4ym3tx atm_cs_qo5vgd atm_9s_1txwivl atm_h_1h6ojuz atm_fc_1h6ojuz atm_bb_idpfg4 atm_3f_glywfm atm_5j_1ssbidh atm_26_1j28jx2 atm_7l_18pqv07 atm_vy_1vi7ecw atm_e2_1vi7ecw atm_gi_idpfg4 atm_gz_logulu atm_h0_logulu atm_l8_idpfg4 atm_uc_1dtz4sb atm_kd_glywfm atm_uc_glywfm__p88qr9 atm_26_1nh1gcj_1nos8r_uv4tnr atm_tr_kv3y6q_csw3t1 atm_26_1nh1gcj_csw3t1 atm_9j_73adwj_1o5j5ji atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_uc_x37zl0_1w3cfyq atm_26_1nh1gcj_1w3cfyq atm_70_1o9v3ru_1w3cfyq atm_uc_glywfm_1w3cfyq_p88qr9 atm_uc_x37zl0_18zk5v0 atm_26_1nh1gcj_18zk5v0 atm_70_1o9v3ru_18zk5v0 atm_uc_glywfm_18zk5v0_p88qr9 dir dir-ltr'})
    for element in elements:
        if element.get_text() == str(page):
            print('\r' + "Found page url: ", page,flush = True, end = "")
            link = 'https://www.airbnb.com.vn' + element.get('href')
            return link
    print('\r' + "Not found page: ", page,flush = True, end = "")
    return None
      

In [6]:
def get_all_page(url):
    pages = []
    pages.append(url)
    pre_page = url
    for i in range(2,16):
        cur_page = move_page(pre_page, i)
        if cur_page:
            pre_page = cur_page
            pages.append(cur_page)
        else:
            break
        time.sleep(2)
        
    return pages


In [7]:
def extract_page(url):
    source = download_html(url)
    soup = BeautifulSoup(source,'lxml')
    houses = soup.find_all('meta',{'itemprop':'url'})
    links = []
    for link in houses:
        URL = 'https://' + link.get('content')
        links.append(URL)
    return links

In [8]:
def get_all_house(url):
    lists = []
    pages = get_all_page(url)
    for page in pages:
        houses = extract_page(page)
        lists.extend(houses)
    return lists

In [9]:
def get_source(url):
    driver = get_driver(url,40)
    for i in range(10):
        try:
            button = driver.find_element(By.XPATH,"//button[@class = 'l1ovpqvx atm_1y33qqm_1ggndnn_10saat9 atm_17zvjtw_zk357r_10saat9 atm_w3cb4q_il40rs_10saat9 b65jmrv atm_9j_tlke0l atm_9s_1o8liyq atm_gi_idpfg4 atm_mk_h2mmj6 atm_r3_1h6ojuz atm_rd_glywfm atm_3f_97hwo atm_vy_1wugsn5 atm_tl_1gw4zv3 atm_9j_13gfvf7_1o5j5ji v7aged4 atm_bx_1ltc5j7 atm_c8_8ycq01 atm_g3_adnk3f atm_cs_qo5vgd atm_5j_9l7fl4 atm_6h_t94yts atm_66_nqa18y atm_uc_1c5iv21 atm_kd_glywfm atm_4b_18pqv07 atm_26_1hbpp16 atm_7l_18pqv07 atm_l8_1vkzbvs atm_uc_glywfm__p88qr9 atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_uc_x37zl0_1w3cfyq atm_uc_glywfm_1w3cfyq_p88qr9 atm_uc_x37zl0_18zk5v0 atm_uc_glywfm_18zk5v0_p88qr9 atm_tr_18md41p_csw3t1 atm_k4_kb7nvz_1o5j5ji atm_4b_18pqv07_1w3cfyq atm_7l_18pqv07_1w3cfyq atm_70_216vci_1w3cfyq atm_4b_18pqv07_18zk5v0 atm_7l_18pqv07_18zk5v0 atm_70_216vci_18zk5v0 atm_4b_1otlplk_1nos8r atm_26_1nh1gcj_1nos8r atm_7l_18pqv07_1nos8r atm_4b_1otlplk_csw3t1 atm_26_1nh1gcj_csw3t1 atm_7l_18pqv07_csw3t1 atm_4b_161hw1_1o5j5ji atm_26_1hbpp16_1o5j5ji atm_7l_161hw1_1o5j5ji dir dir-ltr']")
            driver.execute_script("arguments[0].click();", button)
            break
        except:
            pass
        
    time.sleep(10)
    pageSource = driver.page_source

    driver.quit()

    return pageSource


In [10]:

def extract_amenities(soup):
    amenities = soup.find_all('div', {'class': '_11jhslp'})
    
    amenities_dict = {}
    for amenity in amenities:
        header = amenity.find('h3', {'class': 'hpipapi atm_7l_1kw7nm4 atm_c8_1x4eueo atm_cs_1kw7nm4 atm_g3_1kw7nm4 atm_gi_idpfg4 atm_l8_idpfg4 atm_kd_idpfg4_pfnrn2 dir dir-ltr'}).get_text()
        values = amenity.find_all('div', {'class': 'f16mr5nx atm_9s_1txwivl dir dir-ltr'})
        values = [v.find(string=True) for v in values]
        
        amenities_dict['amenity_' + header] = values
    return amenities_dict
        


In [11]:

def extract_ratings(soup):
    total = soup.find('span',{'class':'t1rl3yjt atm_9s_116y0ak dir dir-ltr'})
    ratings = soup.find_all('div',{'class':'l925rvg atm_9s_1txwivl atm_ar_1bp4okc atm_cx_yh40bf dir dir-ltr'})
    ratings_dict = {}
    ratings_dict['rating_Total'] = total.find('div',{'dir':'ltr'}).get_text()
    for rating in ratings:
        header = rating.contents[0].get_text()
        values = rating.contents[1].get_text()
        
        ratings_dict['rating_' + header] = values
    return ratings_dict
        


In [12]:

def extract_prices(soup):
    try:
        price_per_night = soup.find('span',{'class':'_tyxjp1'}).get_text().replace('\xa0', '')
    except:
        price_per_night = soup.find('span',{'class':'_1y74zjx'}).get_text().replace('\xa0', '')

    
    prices = soup.find_all('div',{'class':'_tr4owt'})
    prices = prices[1:]
    prices_dict = {}
    prices_dict['price_per_night'] = price_per_night
    for price in prices:
        header = price.find('div',{'class':'_m6lwl6'}).get_text()
        try:
            values = price.find('span',{'class':'_1k4xcdh'}).get_text()
        except:
            values = price.find('span',{'class':'_1rc8xn5'}).get_text()

        prices_dict['prices_' + header] = values
    return prices_dict

In [13]:
import json 

def scrape_house(url):
    source = get_source(url)
    soup = BeautifulSoup(source, 'html.parser')
    air_bnb_data = {}

    try:
        air_bnb_data['url'] = url
    except:
        air_bnb_data['url'] = None
        
    try:
        air_bnb_data['title'] = soup.find('span',{'class':'_1xxgv6l'}).get_text()
    except:
        air_bnb_data['title'] = None
    
    try:
        air_bnb_data['bed'] = soup.find('ol',{'class':'lgx66tx atm_gi_idpfg4 atm_l8_idpfg4 dir dir-ltr'}).get_text()
    except:
        air_bnb_data['bed'] = None
    try:
        air_bnb_data['title2'] = soup.find('div',{'class':'toieuka atm_c8_8ycq01 atm_g3_adnk3f atm_fr_rvubnj atm_cs_qo5vgd atm_c8_8nb4eg__oggzyc atm_g3_1dpnnv7__oggzyc atm_fr_11dsdeo__oggzyc dir dir-ltr'}).get_text()
    except:
        air_bnb_data['title2'] = None
    try:
        location = soup.find('div',{'class':'_leqb4t'})
        if location:
            air_bnb_data['location'] = location.get_text()
        else:
            air_bnb_data['location'] = soup.find('div',{'class':'_152qbzi'}).get_text()
    except:
        air_bnb_data['location'] = None
    
    try:
        air_bnb_data['badge'] = soup.find('div',{'class':'lbjrbi0 atm_le_1y44olf atm_lk_1y44olf atm_ll_1y44olf dir dir-ltr'}).get_text()
    except:
        air_bnb_data['badge'] = None        
    try:
        air_bnb_data['cancellation_policy'] = soup.find('div',{'class':'tutyap8 atm_gq_xvenqj dir dir-ltr'}).get_text()
    except:
        air_bnb_data['cancellation_policy'] = None   
    try:
        rules = soup.find_all('div', {'class': 'i1303y2k atm_h_1q9ycp6 atm_9s_1txwivl atm_gq_logulu dir dir-ltr'})
        pattern = re.compile('.*khách.*')

        for rule in rules:
            string = rule.get_text()  # Lấy nội dung văn bản trong mỗi div
            if re.search(pattern, string):  # Tìm kiếm chuỗi thỏa mãn biểu thức chính quy
                accommodates = string
                break 
        air_bnb_data['accommodates'] = accommodates
    except:
        air_bnb_data['accommodates'] = None   
    try:
        amenities = extract_amenities(soup)
        air_bnb_data.update(amenities)
    except:
        pass

    try:
        ratings = extract_ratings(soup)
        air_bnb_data.update(ratings)
    except:
        pass
    try:
        prices = extract_prices(soup)
        air_bnb_data.update(prices)
    except:
        pass
    return json.dumps(air_bnb_data)

In [14]:
def scrape_all_house(urls):
    house_list = []
    for house in urls:
        time.sleep(1)
        try:
            house_data = scrape_house(house)
        except:
            pass
        house_list.append(house_data)
    return house_list


In [15]:
# CPU intensive process -> use multiprocessing :)
from multiprocessing import Pool

In [16]:
# typically we could set "n" to the number of cpu's
import os
n_cpu = os.cpu_count()

In [17]:
urls = ['https://www.airbnb.com.vn/s/B%C3%ACnh-Th%E1%BA%A1nh--Ho-Chi-Minh-City--Vietnam/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-01-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&query=B%C3%ACnh%20Th%E1%BA%A1nh%2C%20Ho%20Chi%20Minh%20City&place_id=ChIJ6eqoAp8odTER481V4JCd7DQ&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_clickextract_page(url)',
       'https://www.airbnb.com.vn/s/Th%E1%BB%A7-%C4%90%E1%BB%A9c--Ho-Chi-Minh--Vietnam/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-01-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&query=Th%E1%BB%A7%20%C4%90%E1%BB%A9c%2C%20Ho%20Chi%20Minh&place_id=ChIJS_ArBF7YdDERTTlkFuy6Jrs&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click',
       'https://www.airbnb.com.vn/s/Qu%E1%BA%ADn-7--Ho-Chi-Minh--Vietnam/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-01-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&query=Qu%E1%BA%ADn%207%2C%20Ho%20Chi%20Minh&place_id=ChIJk7BQsGAldTERycxVEMWJy20&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click',
       'https://www.airbnb.com.vn/s/Qu%E1%BA%ADn-1--Ho-Chi-Minh--Vietnam/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-01-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&query=Qu%E1%BA%ADn%201%2C%20Ho%20Chi%20Minh&place_id=ChIJe4jt-TgvdTERiYl2A1ftrRQ&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click',
       'https://www.airbnb.com.vn/s/Qu%E1%BA%ADn-5/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-01-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change']


In [18]:
# t0 = time.time()
# detail_features = scrape_house([0])
# print(time.time() - t0)

In [19]:
def scrape_all(url):
    listing_urls = get_all_house(url)
    for n_pools in [8]:
        with Pool(n_pools) as pool:
            result = list(pool.map(scrape_house, listing_urls))

        pool.close()
        pool.join()
    return [json.loads(house) for house in result]


In [20]:
data = scrape_all(urls[3])

Found page url:  15

In [21]:
# lists = []
# for url in urls:
#     for i in range(10):
#         try:
#             result = scrape_all(url)
#             lists.append(result)
#             break
#         except:
#             pass

In [26]:
# data = [json.loads(house) for house in result]
data = pd.DataFrame(data)
data.to_csv('q1.csv')

In [29]:
# data = [json.loads(house) for house in result]
data = pd.DataFrame(data)
data.to_csv('q5.csv')

Found page url:  15

In [31]:
# data = [json.loads(house) for house in result]
data = scrape_all(urls[0])
data = pd.DataFrame(data)
data.to_csv('bt.csv')

Found page url:  15

In [32]:
import pandas as pd

# Đọc các file CSV
df1 = pd.read_csv('thuduc.csv')
df2 = pd.read_csv('bt.csv')
df3 = pd.read_csv('q1.csv')
df4 = pd.read_csv('q7.csv')
df5 = pd.read_csv('q5.csv')


# Gộp DataFrame lại với nhau
combined = pd.concat([df1, df2,df3,df4,df5])

# Lưu kết quả vào file mới
combined.to_csv('dataset.csv', index=False)


## Preliminar Data Engineering for the data extracted by web crawler



### Import libraries

In [61]:
data = pd.read_csv('dataset.csv')

In [62]:
import re
import pandas as pd
import numpy as np


In [63]:
data.drop(data.columns[0], axis=1, inplace=True)
data.fillna("", inplace = True)
# data.drop(columns = ['url'], inplace = True)

In [64]:
def make_rooms(data):
    data['rooms_split'] = data['bed'].str.split(' ·  · ')

    def extract_integer(x):
        if isinstance(x, list) and len(x) > 0:
            try:
                return int(x[0].split(' ')[0])
            except ValueError:
                return None
        return None

    data['cnt_guests'] = data['rooms_split'].apply(lambda x: extract_integer(x))
    data['cnt_bedrooms'] = data['rooms_split'].apply(lambda x: extract_integer(x[1:]) if isinstance(x, list) and len(x) > 1 else None)
    data['cnt_beds'] = data['rooms_split'].apply(lambda x: extract_integer(x[2:]) if isinstance(x, list) and len(x) > 2 else None)
    data['cnt_baths'] = data['rooms_split'].apply(lambda x: extract_integer(x[3:]) if isinstance(x, list) and len(x) > 3 else None)
    
    return data

In [65]:
def make_ratings_reviews(data):
    data['total_rating'] = data['rating_Total'].str.split(' · ').apply(lambda x: x[0].replace(',','.') if isinstance(x, list) and len(x) > 1 else None)
    data['total_reviews'] = data['rating_Total'].str.split(' · ').apply(lambda x: x[1].split(' ')[0] if isinstance(x, list) and len(x) > 1 else None)
    return data

In [66]:
def make_prices(data):
    data['price_per_night'] = data['price_per_night'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else None)
    data['prices_Phí dịch vụ Airbnb'] = data['prices_Phí dịch vụ Airbnb'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else 0)
    data['prices_Phí vệ sinh'] = data['prices_Phí vệ sinh'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else 0)
    data['prices_Ưu đãi đặc biệt'] = data['prices_Ưu đãi đặc biệt'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else 0)
    data['prices_Phí vệ sinh'] = data['prices_Phí vệ sinh'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else 0)
    data['prices_Giảm giá cho thời gian ở dài hạn'] = data['prices_Giảm giá cho thời gian ở dài hạn'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else 0)


    return data

In [67]:
def make_cancellation_policy(data):
    rules = ['miễn phí', 'một phần', 'không được']
    return_rules = ['free', 'part', 'no refund']
    
    def map_rules(text):
        if isinstance(text, str):
            for index, rule in enumerate(rules):
                if re.search(rule, text, flags=re.IGNORECASE):
                    return return_rules[index]
        return None
    
    data['cancellation_policy'] = data['cancellation_policy'].apply(map_rules)
    return data


In [68]:
def make_room_types(data):
    rules = ['phòng riêng', 'phòng', 'căn hộ']
    default_value = 'khác'
    data['title2'] = data['title2'].replace("",np.nan).fillna(data['title'])

    # Function to map room types based on title
    def map_room_type(text):
        if isinstance(text, str):
            for rule in rules:
                if rule == 'phòng riêng':
                    # Handling 'phòng riêng' case separately
                    match = re.search(r'\bphòng riêng\b', text, flags=re.IGNORECASE)
                    if match:
                        return 'phòng riêng'
                else:
                    match = re.search(rule, text, flags=re.IGNORECASE)
                    if match:
                        return rule
        return default_value

    data['room_type'] = data['title2'].apply(map_room_type)
    return data


In [69]:
def make_clean(data):
  data = make_rooms(data)
  data = make_ratings_reviews(data)
  data = make_prices(data)
  data = make_cancellation_policy(data)
  data = make_room_types(data)

  data['accommodates'] = data['accommodates'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else None)
  data['accommodates'] = data['accommodates'].replace("",np.nan).fillna(data['cnt_guests'])
  return data


In [70]:
data = make_clean(data)

data.to_csv('data.csv')
